# Setup

In [4]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.lvilla17.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.35"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.lvilla17.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [5]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [6]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.lvilla17.vpn.itam.mx:9041', 'cassandra-node-2.lvilla17.vpn.itam.mx:9042', 'cassandra-node-3.lvilla17.vpn.itam.mx:9043']
JDBC URL: jdbc:cassandra://cassandra-node-1.lvilla17.vpn.itam.mx:9041,cassandra-node-2.lvilla17.vpn.itam.mx:9042,cassandra-node-3.lvilla17.vpn.itam.mx:9043
✅ Connected to cluster: cassandra-cluster.lvilla17.vpn.itam.mx
🌐 Nodes found: 3


In [7]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-3.lvilla17.vpn.itam.mx:9043,10.15.20.35:9041,10.15.20.35:9042
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x10e65f1d0>,<cassandra.policies.SimpleConvictionPolicy object at 0x10e6daf10>,<cassandra.policies.SimpleConvictionPolicy object at 0x10e6db240>
host_id,f0a245bb-0eb9-4609-a57f-afb813c53940,e4858b9a-7523-467d-891b-5fa198a15965,54f480b7-4753-405a-bd59-d186b4d850d5
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x10fd5f080>,<unlocked _thread.RLock object owner=0 count=0 at 0x10fd2c300>,<unlocked _thread.RLock object owner=0 count=0 at 0x10e64da40>
is_up,True,True,True
listen_address,172.19.0.4,NaN,NaN
listen_port,7003.0,NaN,NaN
broadcast_address,10.15.20.35,10.15.20.35,10.15.20.35


In [8]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [9]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [10]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [11]:
from faker import Faker

fake = Faker()

In [12]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [13]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [14]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Ericstad', 'user_id': UUID('aba16fa5-a7f2-4a85-83e0-3a5d181ff3fc'), 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 979000), 'session_duration': 2328, 'username': 'pcameron'}
{'city': 'Port Michaelhaven', 'user_id': UUID('b28213bf-7b00-4ed2-b40a-e71c37dd2beb'), 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 984000), 'session_duration': 661, 'username': 'clarkjacob'}
{'city': 'Lifort', 'user_id': UUID('8f4f262f-f225-4a07-8c0d-a193154f7c90'), 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 981000), 'session_duration': 582, 'username': 'gonzalezthomas'}
{'city': 'Williamsonshire', 'user_id': UUID('08149c58-5f3f-4e84-ae1e-767aaf99fd1a'), 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 991000), 'session_duration': 740, 'username': 'kevinhess'}
{'city': 'Kevinland', 'user_id': UUID('503a2257-3f51-44f5-a788-3b6ceca9aae1'), 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 980000), 'session_duration': 747, 'username': 'rangelheather'}


In [15]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Ericstad'}
{'city': 'Port Michaelhaven'}
{'city': 'Lifort'}
{'city': 'Williamsonshire'}
{'city': 'Kevinland'}
{'city': 'East Nathan'}
{'city': 'New Victorview'}
{'city': 'Bankschester'}
{'city': 'West Michaelside'}
{'city': 'Davisland'}
{'city': 'Scottmouth'}
{'city': 'New Kathleen'}
{'city': 'New Deborahfurt'}
{'city': 'Pierceshire'}
{'city': 'Port Adam'}
{'city': 'Andrewville'}
{'city': 'New Christianhaven'}
{'city': 'East Breannaland'}
{'city': 'East Jacqueline'}
{'city': 'North Thomas'}
{'city': 'Christophershire'}
{'city': 'Timothystad'}
{'city': 'Alanbury'}
{'city': 'East Stacey'}
{'city': 'Mandyshire'}
{'city': 'North Tammy'}
{'city': 'Lake Christophermouth'}
{'city': 'Mayberg'}
{'city': 'Port Leslie'}
{'city': 'Lake Cynthia'}
{'city': 'Gilbertberg'}
{'city': 'Lake Omarhaven'}
{'city': 'Nicoleberg'}
{'city': 'South Christopherhaven'}
{'city': 'West Emily'}
{'city': 'Gonzalezview'}
{'city': 'Espinozaport'}
{'city': 'Port David'}
{'city': 'West Josephton'}
{'city': 'Carl

### Find nodes storing random data

In [16]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Lake Scott',
 'last_access': datetime.datetime(2026, 1, 26, 21, 58, 26, 983000),
 'session_duration': 1722,
 'user_id': UUID('260ec6ff-9e0f-4ccd-aa88-a36d7a4908d8'),
 'username': 'andersensandra'}
Nodes storing 'Lake Scott':
 - Host: cassandra-node-3.lvilla17.vpn.itam.mx, Gossip Port: 7003
 - Host: 10.15.20.35, Gossip Port: 7001
 - Host: 10.15.20.35, Gossip Port: 7002


### ORM-like

In [17]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [18]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=South Faithfort, user_id=8ad652ed-9dce-438a-9345-5e0aca223335>
Query user_metris: UserMetrics <city=Ericstad, user_id=aba16fa5-a7f2-4a85-83e0-3a5d181ff3fc>
Query user_metris: UserMetrics <city=Port Michaelhaven, user_id=b28213bf-7b00-4ed2-b40a-e71c37dd2beb>
Query user_metris: UserMetrics <city=Lifort, user_id=8f4f262f-f225-4a07-8c0d-a193154f7c90>
Query user_metris: UserMetrics <city=Williamsonshire, user_id=08149c58-5f3f-4e84-ae1e-767aaf99fd1a>
Query user_metris: UserMetrics <city=Kevinland, user_id=503a2257-3f51-44f5-a788-3b6ceca9aae1>
Query user_metris: UserMetrics <city=East Nathan, user_id=89821848-098b-4e2c-baef-aabf768ad9bd>
Query user_metris: UserMetrics <city=East Nathan, user_id=9e93530a-e68b-4c70-a672-7d61c795ec09>
Query user_metris: UserMetrics <city=New Victorview, user_id=1a5732dd-7a3c-4a19-9641-b8ed709c8fa1>
Query user_metris: UserMetrics <city=Bankschester, user_id=3c606f55-a22a-4696-8008-c30d10420620>
Query user_metris: UserMetrics <

### Insert/Create

In [19]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Jamesfort', user_id=UUID('799fc924-016f-4061-90eb-fd51a4cc82d5'), username='cmarshall', session_duration=972, last_access=datetime.datetime(2010, 6, 25, 23, 10, 58, 813954))

In [20]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [21]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Ericstad, user_id=aba16fa5-a7f2-4a85-83e0-3a5d181ff3fc>
Delete user_metris: UserMetrics <city=Port Michaelhaven, user_id=b28213bf-7b00-4ed2-b40a-e71c37dd2beb>
Delete user_metris: UserMetrics <city=Lifort, user_id=8f4f262f-f225-4a07-8c0d-a193154f7c90>
Delete user_metris: UserMetrics <city=Crawfordview, user_id=ede319fe-5c37-43bf-93ed-07e159b042cb>
Delete user_metris: UserMetrics <city=Williamsonshire, user_id=08149c58-5f3f-4e84-ae1e-767aaf99fd1a>
Delete user_metris: UserMetrics <city=Kevinland, user_id=503a2257-3f51-44f5-a788-3b6ceca9aae1>
Delete user_metris: UserMetrics <city=East Nathan, user_id=89821848-098b-4e2c-baef-aabf768ad9bd>
Delete user_metris: UserMetrics <city=East Nathan, user_id=9e93530a-e68b-4c70-a672-7d61c795ec09>
Delete user_metris: UserMetrics <city=New Victorview, user_id=1a5732dd-7a3c-4a19-9641-b8ed709c8fa1>
Delete user_metris: UserMetrics <city=Bankschester, user_id=3c606f55-a22a-4696-8008-c30d10420620>
Delete user_metris: UserM

In [22]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")